# Публичные zip-codes для Бразилии

Предназначение этого ноутбука: совместить zip-codes с местоположением. 

In [ ]:
import pandas as pd
import unicodedata
from olist_churn_prediction.paths import SRC_DIR, PROCESSED_DIR, INTERIM_NOTEBOOK_DIR
from olist_churn_prediction import feature_processing, feature_engineering

In [14]:
zip_codes = pd.read_csv(r"C:\BR.txt", header=None, sep="\t")

In [15]:
zip_codes.columns = ['0', 'zip_code', 'place_name', 'state', 'state_code', 'city', 'city_code', 'admin_name_3', 'admin_code_3', 'lat', 'lng', 'accuracy']

In [16]:
zip_codes = zip_codes.drop(['0', 'place_name', 'admin_name_3', 'admin_code_3', 'state_code', 'accuracy', 'city_code'], axis=1)

In [17]:
zip_codes.head()

,zip_code,state,city,lat,lng
0,69945-000,Acre,Acrelândia,-9.9805,-66.8439
1,69935-000,Acre,Assis Brasil,-10.8833,-70.0131
2,69932-000,Acre,Brasiléia,-10.7677,-69.0114
3,69923-000,Acre,Bujari,-9.5786,-68.1720
4,69922-000,Acre,Capixaba,-10.4878,-67.8483


In [53]:
#zip_codes.to_parquet(r"C:\Users\Вячеслав\Documents\olist_churn\olist-ml-churn-prediction\data\raw\zip_codes.parquet")

In [8]:
public = pd.read_parquet(r"C:\Users\Вячеслав\Documents\olist_churn\olist-ml-churn-prediction\data\interim\notebook_related\public_data_interim.parquet")

In [24]:
zip_codes = zip_codes.dropna()

In [25]:
print(zip_codes.isna().sum())

zip_code    0
state       0
city        0
lat         0
lng         0
dtype: int64


In [26]:
def normalize_text(text: str) -> str:
    text = unicodedata.normalize('NFKD', text)
    text = ''.join([c for c in text if not unicodedata.combining(c)])
    return text

# Применение к колонке
zip_codes['city_normalized'] = zip_codes['city'].apply(normalize_text)

       zip_code      state            city      lat      lng city_normalized
0     69945-000       Acre      Acrelândia  -9.9805 -66.8439      Acrelandia
1     69935-000       Acre    Assis Brasil -10.8833 -70.0131    Assis Brasil
2     69932-000       Acre       Brasiléia -10.7677 -69.0114       Brasileia
3     69923-000       Acre          Bujari  -9.5786 -68.1720          Bujari
4     69922-000       Acre        Capixaba -10.4878 -67.8483        Capixaba
...         ...        ...             ...      ...      ...             ...
5519  77900-000  Tocantins  Tocantinópolis  -6.2625 -47.5577  Tocantinopolis
5520  77704-000  Tocantins        Tupirama  -8.9192 -48.2766        Tupirama
5521  77743-000  Tocantins      Tupiratins  -8.3858 -48.2230      Tupiratins
5522  77860-000  Tocantins    Wanderlândia  -6.7980 -48.0041    Wanderlandia
5523  77880-000  Tocantins         Xambioá  -6.5941 -48.4314         Xambioa

[5495 rows x 6 columns]


In [28]:
zip_codes = zip_codes.drop("city", axis=1)

In [30]:
cat_cols=['city_normalized', '']

In [31]:
feature_processing.lowercase_categoricals(zip_codes, cat_cols=cat_cols, inplace=True)

,zip_code,state,lat,lng,city_normalized
0,69945-000,Acre,-9.9805,-66.8439,acrelandia
1,69935-000,Acre,-10.8833,-70.0131,assis_brasil
2,69932-000,Acre,-10.7677,-69.0114,brasileia
3,69923-000,Acre,-9.5786,-68.1720,bujari
4,69922-000,Acre,-10.4878,-67.8483,capixaba
...,...,...,...,...,...
5519,77900-000,Tocantins,-6.2625,-47.5577,tocantinopolis
5520,77704-000,Tocantins,-8.9192,-48.2766,tupirama
5521,77743-000,Tocantins,-8.3858,-48.2230,tupiratins
5522,77860-000,Tocantins,-6.7980,-48.0041,wanderlandia


In [32]:
zip_codes['zip_code'] = zip_codes['zip_code'].str[:3]

In [35]:
zip_codes['city_normalized'].value_counts()

city_normalized
sao_domingos      5
bom_jesus         5
bonito            4
planalto          4
santa_ines        4
                 ..
tocantinopolis    1
tupirama          1
tupiratins        1
wanderlandia      1
brasileia         1
Name: count, Length: 5219, dtype: int64

In [34]:
zip_codes.dtypes

zip_code            object
state               object
lat                float64
lng                float64
city_normalized     object
dtype: object

Пришлось отказаться от идеи получить zip_codes для продавцов для создания новых признаков, основанных на расстоянии между точкой отправки товара и точкой получения. Возможно, данная возможность появится позже.